## lmppl를 사용한 perplexity 계산

#### https://github.com/asahi417/lmppl

In [ ]:
!pip install lmppl

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
  Created wheel for lmppl: filename=lmppl-0.3.2-py3-none-any.whl size=13487 sha256=3e3639e336858f110e060f186c9fa99ea3fde3f36546d2a1b78dd5705810265d
  Stored in directory: /root/.cache/pip/wheels/7f/a5/ab/9211abbfb49e56f34266e26aae45218f09b6fa142951bcff88
Successfully built lmppl
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-aiplatform 1.71.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!

In [ ]:
!pip install protobuf==3.20.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
lmppl 0.3.2 requires protobuf<3.20, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
import lmppl

# GPT-2 모델 불러오기
scorer = lmppl.LM('gpt2')

# Shakespeare 텍스트 파일 읽기 (전체 텍스트를 읽어 text에 저장)
with open('/content/shakespeare.txt', 'r') as file:
    text = file.read()

# Perplexity 계산 (리스트가 아닌 단일 텍스트로 계산)
ppl = scorer.get_perplexity([text])  # text를 리스트로 감싸야 메서드가 잘 동작합니다.

# 결과 출력
print(f"Perplexity of the entire text: {ppl[0]}")
#텍스트를 처음부터 끝까지 읽으며 각 단어의 등장 확률을 계산해 perplexity를 구합니다.

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1002: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
100%|██████████| 1/1 [00:14<00:00, 14.25s/it]


Perplexity of the entire text: 64.07899842177692


In [ ]:
import lmppl

# Encoder-DecoderLM 모델 불러오기
scorer = lmppl.EncoderDecoderLM('google/flan-t5-small')

# Shakespeare 텍스트 파일 읽기 (전체 텍스트를 읽어 input과 output으로 나누기)
with open('/content/shakespeare.txt', 'r') as file:
    text = file.read()

# 예시로 전체 텍스트를 입력으로 사용하고 같은 내용을 출력으로 설정
# 입력과 출력이 같은 경우 전체 문장 연결성을 기반으로 perplexity를 평가할 수 있습니다.
inputs = [text]
outputs = [text]

# Perplexity 계산 (리스트로 감싸서 전달)
ppl = scorer.get_perplexity(input_texts=inputs, output_texts=outputs)

# 결과 출력
print(f"Perplexity of the entire text: {ppl[0]}")

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
100%|██████████| 1/1 [00:14<00:00, 14.99s/it]


Perplexity of the entire text: 1.0790774293635352


#### 입력 텍스트에 따라 자연스러운 타켓 텍스트를 예측하는 작업은 Flan-T5, DeBERTa, 일반적인 문장의 자연스러움 평가는 GPT-2가 더 적합함.

## BERT 기반 Masked Language Model 아키텍쳐 사용시 Perplexity

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

In [ ]:
model_name = 'cointegrated/rubert-tiny' #모델을 MLM형태로 불러옴.
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name) #문장을 토큰화하여, 모델의 입력 형식에 맞게 문장을 숫자 토큰으로 변환.

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt') #문장을 토크나이저로 BERT 모델에 적합한 토큰 텐서 형태로 인코딩
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1) #문장에서 각 단어를 차례로 마스킹하기 위해 문장의 토큰 텐서를 여러 번 반복해서 생성
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2] #각 단어를 순차적으로 마스킹하도록 마스크를 생성
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100) #마스킹된 위치에 원래의 단어 토큰을 남기고, 그 외의 위치에는 손실 계산에서 제외되도록 -100으로 설정.
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item()) #모델이 마스킹된 문장 조합에서 원래 단어를 얼마나 정확하게 예측하는지를 기반으로 손실을 계산. 낮을수록 모델이 해당 문장을 더 자연스럽게 인식

In [ ]:
print(score(sentence='London is the capital of Great Britain.', model=model, tokenizer=tokenizer))
# 4.541251105675365
print(score(sentence='London is the capital of South America.', model=model, tokenizer=tokenizer))
# 6.162017238332462

34.74946336845598
6.162018707472031
